In [ ]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
# from pandas import DataFrame
import requests
import json
import re
import sys
import pandas as pd

In [ ]:
#Prep for looping over all pages.
i=0
url = "https://www.ictergezocht.nl/it-vacatures/?pnr=" + str(i)

#Creating storage for URLS
links1 = [] #Create open list to store links later. non spot
links2 = [] #Create open list to store links later. spotlight vacancies
links3 = [] #Both attempt <- Works!



while True:
    i = i+1
    if i == 75: #make sure it stops at last page. 
        break

    #should stop the code in case of error
    page = requests.get(url)
    if page.status_code != 200:
        break

    url = "https://www.ictergezocht.nl/it-vacatures/?pnr=" + str(i)
    print(url)
    #Data retrieval prepetaion
    uClient = uReq(url)
    page_html = uClient.read()
    uClient.close()

    page_soup1 = soup(page_html,"html.parser")

    #Grabs each block (spot and non spot)
    content_blocks = page_soup1.findAll("div",{"class":"content_block vacitem"})

    spot_content_blocks = page_soup1.findAll("div",{"class":"content_block vacitem spotlight "})

    #Grab specific info for each block
    aaa = 0
    for content_block in content_blocks:
        aaa = aaa + 1
        
        t = content_block.find("a",{"class":"title"})
        try:
            link = t.get('href')
        except Exception:
            pass

        #title_block = content_block.findAll("a",{"class":"title"})
        #func_title = title_block[0].text

        links1.append(link) #My attempt to save everything in the same list
        links3.append(link) #Attempt all
        print(aaa)
        #print("Job title: "+ func_title)
        #print(link)


    
    for spot_content_block in spot_content_blocks:
        aaa = aaa +1
        t = spot_content_block.find("a",{"class":"title"})
        try:
            link = t.get('href')
        except Exception:
            pass
        
        
        #spot_title_block = spot_content_block.findAll("a",{"class":"title"})
        #spot_func_title = spot_title_block[0].text

        links2.append(link) #My attempt to save everything in the same list
        links3.append(link) #Attempt all
        print(aaa)
        #print("Spotlight Job title: "+ spot_func_title)
        #print(link)


#############################################################################
print("DONE stage1! Retrieving Links to Job descriptions")
#############################################################################

In [ ]:
print(len(links1), len(links2), len(links3))

In [ ]:
#Preparing CSV
ict_old = pd.read_csv("ict-vacatures-text-all.csv")
filename = "placeholder"
topics_dict = { "Job_title":[], \
                "Company":[], \
                "Location":[], \
                "Offer":[], \
               "Link":[], \
                "Posted":[], \
                "Text":[], \
                }


urls = links3
test_list = []
for url in urls:
    print(url)
    uClient = uReq(url)
    page_html = uClient.read()
    uClient.close()

    vac_soup1 = soup(page_html,"html.parser")
    data = vac_soup1.find("script", type="application/ld+json").text
    data = ' '.join(re.sub(r"(\n|\t)*", "", data).split()) # --->>> REMOVE NEW LINE CHARS AND WHITESPACES

    # --->>> PREPROCESSING DATA TO REPLACE HTML CODES WITH SIGNS
    data = re.sub(r"\&amp;", "&", data) # --->>> AMPERSAND '&'
    data = re.sub(r"\&reg;", "", data) # --->>> REGISTERED SIGN '®'

    # --->>> IMPORTANT <<<--- #
    # --->>> APPARENTLY SOUP MAKES SOMETIMES A MISTAKE WITH WRITING ANOTHER COMMA AT THE END OF A BLOCK
    # --->>> WHERE NO MORE BLOCKS ARE COMING AND JSON DOES NOT LIKE THAT...
    data = re.sub(r"\},\s\}", "} }", data) # --->>> REMOVE TRAILING COMMA IF NO MORE BLOCKS ARE COMING
    
    # --->>> NEW STEP <<<--- #
    # --->>> USE A LOOKBEHIND AND A LOOKAHEAD AROUND A COMMA.
    # --->>> IF THEY MATCH NUMBER COMMA NUMBER.
    # --->>> REPLACE IT WITH NUMBER DOT NUMBER
    # --->>> THERE ARE ALSO NO QUANTIFIERS POSSIBLE IN THE LOOKAHEAD/LOOKBEHIND
    data = re.sub(r"(?<=\d),(?=\d)", ".", data) # --->>> REPLACE A COMMA BETWEEN 2 NUMBERS WITH A DOT
    
    # --->>> END PREPROCESSING

    # --->>> I USED THIS TO FIND THE ERRORS AT RUNTIME
    try:
        newDictionary = json.loads(data) # --->>> LOAD EVERYTHING TO A DICTIONARY
    except (Exception, ArithmeticError) as e:
        template = "An exception of type {0} occurred. Arguments:\n{1!r}" # --->>> ERROR MESSAGE TEMPLATE
        message = template.format(type(e).__name__, e.args) # --->>> COMPILE ERROR MESSAGE
        print (message) # --->>> PRINT THE ERROR
        print(data) # --->>> PRINT DATA TO CHECK WHAT COULD CAUSE ERROR
        sys.exit("JSONDecodeError") # --->>> STOP EXECUTION OF SCRIPT

    # --->>> THE VALUES WE ARE INTERESTED IN
    # --->>> JUST ADD MORE IF YOU NEED
    # --->>> USE print(newDictionary) TO FIND THE INDICES YOU ARE INTERESTED IN
    # print(newDictionary)
    title = newDictionary['title']
    company = newDictionary['hiringOrganization']['name']
    offer = newDictionary['@type']
    location = newDictionary['jobLocation']['address']['addressLocality']
    date = newDictionary['datePosted']

    #Preparing for CSV
    title.replace(",",";")
    company.replace(",",";")
    offer.replace(",",";")
    location.replace(",",";")
    date.replace(",",";")    
    #Writing to Pandas frame
    topics_dict["Job_title"].append(title)
    topics_dict["Company"].append(company)
    topics_dict["Location"].append(location)
    topics_dict["Offer"].append(offer)
    topics_dict["Link"].append(url)
    topics_dict["Posted"].append(date)

    all_text = []
    vactexts = vac_soup1.findAll("div",{"class":"vactext"}) 
    for vactext in vactexts:
        tekst = vactext.text
        tekst = re.sub(r"[\n\t]*", "", tekst)

        all_text.append(tekst)

        str_text = ''.join(all_text) 
        str_text = re.sub(r'[\n\t]',' ', str_text)
    # --->>> END PREPROCESSING

    # --->>> WRITE VALUES TO FILE
    str_text.replace(",",";")
    topics_dict["Text"].append(str_text)



#############################################################################
print("DONE stage2! Retrieving the data")
#############################################################################

In [ ]:
data

In [ ]:
ict_new = pd.DataFrame(topics_dict)
ict_all = ict_old.append(ict_new, sort=False, ignore_index=True)

ict_all.columns = ict_all.columns.str.strip()
ict_all.columns.tolist()

ict_all = ict_all.drop_duplicates(['Link'])
ict_all.to_csv("ict-vacatures-text-all.csv", encoding = "utf-8", index= False)

print(len(ict_new), len(ict_old), len(ict_all), sep='\n')